In [1]:
# read in pandas
import pandas as pd

# define filepath for data
fp = r'/Users/admin/python-environments/Final Assignment/data/2315676.txt'

# Read in data from file
# identify null values and skip the first row of empty data
data = pd.read_csv(fp, sep='\s+', na_values = -9999, skiprows = [1])

# select rows of interest
data = data[["DATE", "TAVG", "TMAX", "TMIN"]]

# create empty dataframe data
data_w_TAVG = pd.DataFrame()

# Where there are null values for the TAVG, calculate the average based on the max and min
# iterate across each for of data
for key, row in data.iterrows():
            
    # determine whether the value of TAVG is NaN
    test = row[["TAVG"]].isna()
    
    # conditional test to see if there is a missing values
    if test["TAVG"] == True:
        
        # if the average temp is missing a value, take the average from TMAX and TMIN
        row["TAVG"] = (row["TMAX"] + row["TMIN"])/2
        
    # for each iteration, create a row of data incluing the update TAVG if it was missing    
    row = row.to_frame().transpose()
          
    # Add each fow to the empty dataframe, data_w_TAVG    
    data_w_TAVG = pd.concat([data_w_TAVG, row], ignore_index = True)
        
# select relevant time frame from 1909 - 2019
# Define start and end date
#start_date = 19090101
#end_date = 20191231

#data_oi = data_w_TAVG.loc[(data_w_TAVG["DATA"] >= start_date) & (data_w_TAVG["DATA"] <= end_date)]
#data_oi.tail()

# convert temps in fahrenheit to to degrees celsius

In [38]:
# Define start and end date
start_date = 19090101
end_date = 20191231

# select the dates of interest from 19090101 - 20191231
data_oi = data_w_TAVG.loc[(data_w_TAVG["DATE"] >= start_date) & (data_w_TAVG["DATE"] <= end_date)]

# define function temperatures in degrees Fahrenheit to degrees celsius
def fahr_to_cels(temp):
    return (temp - 32) * 5/9

# create new columns for temps in Fahrenheit
data_oi = data_oi.assign(TAVG_cels = fahr_to_cels(data_oi["TAVG"]))
data_oi.dtypes

DATE         float64
TAVG         float64
TMAX         float64
TMIN         float64
TAVG_cels    float64
dtype: object

In [48]:
""" Define seasonal averages for summer, autumn, winter, spring based on the following months
        - Winter: December, January, February
        - Spring: March, April, May
        - Summer: June, July, August
        - Autumn: September, October, November

Need to split date into months and years
Then need to convert months into seasons using conditi
Form group based on months
Create a new column which calculates the monthly average temperature from the group
"""      
# Calculate seasonal averages from 1909 - 2019
# Create new date column with just the year and month
data_oi = data_oi.assign(Year = data_oi["DATE"].astype(str).str.slice(start=0, stop=6))

# create group based on each year month combination
grouped = data_oi.groupby('Year_Month')

# create new empty dataframe monthly_data to calculate the monthly average temps
monthly_data = pd.DataFrame()

# use for loop
for key, group in grouped:
    
    # calculate monthly average
    mean_values = group[["TAVG_cels"]].mean()
    
    # add Year_Month column to 'mean_values'
    mean_values["Year_Month"] = key
    
    # convert data into a row entry to support input into dataframe
    row = mean_values.to_frame().transpose()
    
    # store each average value within a new dataframe called monthly_data
    monthly_data = pd.concat([monthly_data, row], ignore_index = True)
    
monthly_data.head()

# Calculate the seasonal averages in references period 1951 - 1980

,TAVG_cels,Year_Month
0,-8.304598,190901
1,-14.74359,190902
2,-11.537037,190903
3,-5.948276,190904
4,0.797491,190905
